In [1]:
import os
import sys

MODULE_PATH = 'C:\Github\DL_Study\Base'
MODEL_PATH = 'C:\Github\DL_Study\RNN'

sys.path.insert(0, MODULE_PATH)
sys.path.insert(0, MODEL_PATH)

import numpy
from config import *
from attention import *

In [2]:
# configuration setting
def model_config():
    # parameter for LSTM Model
    epochs = [30]
    batch_size = [32]
    learning_rate = [0.01, 0.001]
    
    # create config data
    configs = []
    for i in epochs:
        for j in batch_size:
            for k in learning_rate:
                config = [i, j, k]
                configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config):
    # unpack config
    n_epochs, n_batch, learning_rate = config
    model = AttentionSeq2Seq(time_size=24, hidden_size=64, feature_size=9)
    # fit model and return
    model.fit(train_X=train_X, train_y=train_y, epochs=n_epochs, 
              batch_size=n_batch, learning_rate=learning_rate)
    return model

def MAE_metric(x, t):
    return np.mean(numpy.abs(x-t))

def MSE_metric(x, t):
    return np.mean((x-t)**2)

In [3]:
import pandas as pd
import numpy
import time
from datetime import datetime

np.random.seed(42)
numpy.random.seed(42)

data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00247/'
data_name = 'data_akbilgic.xlsx'
df = pd.read_excel(data_url+data_name, header=1)
df.drop(columns=df.columns[[0]], axis=1, inplace=True)
cols = df.columns.tolist()
cols = cols[3:]+cols[:3]
df = df[cols]
df.head()

C:\Users\lab\AppData\Local\Temp/ipykernel_17460/1979493742.py:11: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df = pd.read_excel(data_url+data_name, header=1)


,DAX,FTSE,NIKKEI,BOVESPA,EU,EM,ISE,ISE.1,SP
0,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524,0.035754,0.038376,-0.004679
1,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773,0.025426,0.031813,0.007787
2,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015,-0.028862,-0.026353,-0.030469
3,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424,-0.062208,-0.084716,0.003391
4,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802,0.009860,0.009658,-0.021533


In [4]:
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

dataset = df.values
dataset = dataset.astype('float')

n_inputs = 24
n_features = 9
del_idx = n_inputs * n_features + 1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)
new_df.head()

,var1(t-24),var2(t-24),var3(t-24),var4(t-24),var5(t-24),var6(t-24),var7(t-24),var8(t-24),var9(t-24),var1(t-23),...,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var1(t)
24,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524,0.035754,0.038376,-0.004679,0.008455,...,0.003932,0.000071,-0.011169,0.024128,-0.004139,0.002073,-0.014133,-0.014571,0.016233,0.029306
25,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773,0.025426,0.031813,0.007787,-0.017833,...,0.029306,0.014788,0.015846,0.039282,0.019127,0.032338,0.036607,0.042759,0.026541,0.004766
26,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015,-0.028862,-0.026353,-0.030469,-0.011726,...,0.004766,0.003651,-0.013411,-0.015462,0.005627,0.007895,0.011353,0.021468,0.001484,-0.035170
27,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424,-0.062208,-0.084716,0.003391,-0.019873,...,-0.035170,-0.022182,-0.002902,-0.021440,-0.024388,-0.002139,-0.040542,-0.043907,-0.050369,0.005434
28,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802,0.009860,0.009658,-0.021533,-0.013526,...,0.005434,0.005019,-0.030745,-0.008799,0.001097,-0.007926,-0.022106,-0.033893,0.007923,-0.027421


In [5]:
n_splits = 10
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs).split(new_df)
next(train_test_split)

configs = model_config()
history = []

best_rmse, best_mse, best_mae = [], [], []
learning_time = []
i = 1

print('config : epochs, batch_size, learning_rate')

# neted cross validation
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = int(len(train_cv_X)*0.2)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1,  n_inputs, n_features)
    val_X = val_X.reshape(-1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1,  n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        model.reset_state()
        predicted = model.predict(val_X)
        if GPU:
            predicted = np.asnumpy(predicted)
        error = np.sqrt(MSE_metric(predicted, val_y))   # rmse
        print(f' error(RMSE):{error}')
        if errors:
            if error < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error)

    history.append(errors)
    
    # check start time
    start_time = time.time()
    # model fitting
    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    # check duration
    duration = time.time()-start_time
    selected_model.reset_state()
    predicted = selected_model.predict(test_cv_X)
    if GPU:
        predicted = np.asnumpy(predicted)

    rmse = np.sqrt(MSE_metric(predicted, test_cv_y))
    mse = MSE_metric(predicted, test_cv_y)
    mae = MAE_metric(predicted, test_cv_y)
    best_rmse.append(rmse)
    best_mse.append(mse)
    best_mae.append(mae)
    learning_time.append(duration)

    # model eval
    print(f'train-size:{train_X.shape[0]}, val-size:{val_X.shape[0]}, test-size:{test_cv_X.shape[0]}')
    print(f'best_model => error(rmse) : {rmse.item():.2f}, param:{configs[param]}, times : {duration:.3f}')
    print()

config : epochs, batch_size, learning_rate
fold : 1/10
 == train [30, 32, 0.01] model ==   error(RMSE):0.10344459359806343
 == train [30, 32, 0.001] model ==   error(RMSE):0.02601538604692319
train-size:55, val-size:13, test-size:42
best_model => error(rmse) : 0.02, param:[30, 32, 0.001], times : 2.849

fold : 2/10
 == train [30, 32, 0.01] model ==   error(RMSE):0.03007675268739116
 == train [30, 32, 0.001] model ==   error(RMSE):0.019275725101278944
train-size:88, val-size:22, test-size:42
best_model => error(rmse) : 0.02, param:[30, 32, 0.001], times : 3.801

fold : 3/10
 == train [30, 32, 0.01] model ==   error(RMSE):0.014758075360434974
 == train [30, 32, 0.001] model ==   error(RMSE):0.017764756003920603
train-size:122, val-size:30, test-size:42
best_model => error(rmse) : 0.01, param:[30, 32, 0.01], times : 5.069

fold : 4/10
 == train [30, 32, 0.01] model ==   error(RMSE):0.014471852904081475
 == train [30, 32, 0.001] model ==   error(RMSE):0.015260518804719077
train-size:156, v

In [6]:
def model_evaluation(mse, rmse, mae):
    mse = np.array(mse)
    rmse = np.array(rmse)
    mae = np.array(mae)
    print(f'MSE: mean={np.mean(mse)}, std={np.std(mse)}')
    print(f'RMSE: mean={np.mean(rmse)}, std={np.std(rmse)}')
    print(f'MAE: mean={np.mean(mae)}, std={np.std(mae)}')

model_evaluation(best_mse, best_rmse, best_mae)

# check time
print()
print('[training time]')
print(f'mean : {np.mean(np.array(learning_time))}, last:{learning_time[-1]}')

MSE: mean=0.00019706575599122284, std=0.00011002061536411078
RMSE: mean=0.01345136481067393, std=0.004015786438717324
MAE: mean=0.010653211883280164, std=0.0033825103490003174

[training time]
mean : 10.209310746192932, last:17.492656707763672
